In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train_raw = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
df_train_raw

## Dataset with annotations

In [ ]:
df_train_raw[df_train_raw["annotations"] != '[]']

## Dataset without annotations

In [ ]:
df_train_raw[df_train_raw["annotations"] == '[]']

## Visualize the images

In [ ]:
from PIL import Image, ImageDraw
import ast

def fetch_image_list(df_tmp, video_id, num_images, start_frame_idx):
    def fetch_image(frame_id):
        path_base = '/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{}/{}.jpg'
        raw_img = Image.open(path_base.format(video_id, frame_id))

        row_frame = df_tmp[(df_tmp.video_id == video_id) & (df_tmp.video_frame == frame_id)].iloc[0]
        bounding_boxes = ast.literal_eval(row_frame.annotations)

        for box in bounding_boxes:
            draw = ImageDraw.Draw(raw_img)
            x0, y0, x1, y1 = (box['x'], box['y'], box['x']+box['width'], box['y']+box['height'])
            draw.rectangle( (x0, y0, x1, y1), outline=180, width=3)
        return raw_img

    return [np.array(fetch_image(start_frame_idx + index)) for index in range(num_images)]

images = fetch_image_list(df_train_raw, video_id = 0, num_images = 80, start_frame_idx = 25)

In [ ]:
print("Num images: ", len(images))
plt.imshow(images[10], interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(9, 9))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

create_animation(images)
